# Insertion des données de DBPedia dans la base de données SQLite locale

### Importer les librairies à utiliser

In [1]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pprint
import csv
import ast

from collections import Counter
from operator import itemgetter

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

import re

In [3]:
### Importer un module de fonctions crée ad hoc
## ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans le même dossier que le présent carnet Jupyter afin que l'importation
#   fonctionne correctement
import sparql_functions as spqf

In [4]:
### Recharger le module après modification des fonctions dans le fichier du module
# désactivé #  reload(spqf)

<module 'sparql_functions' from '/Volumes/Patientia/python_notebooks/sciences_historiques_numeriques/histoire_numerique/sparql/sparql_functions.py'>

### Versionnement manuel de la base de donnée des requêtes

Cette opération créé une copie de la base de données identifiée par un _timestamp_. À utiliser avant toute opération significative sur la base de données (modification de la structure, imports par des scripts Python, etc.)

In [4]:
db = 'data/sparql_queries.db'

In [5]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210513_160213'

In [6]:
### Définir les adresses des fichiers, l'existant et celui à créer
db = 'data/sparql_queries.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/sparql_queries_' + timestamp_formatted_for_file_name() + '.sqlite'

In [7]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afint de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(db, timestamped_db_copy)
copied_db

'data/sparql_queries_20210513_160214.sqlite'

## Exécuter une requête SPARQL stockée dans la base de données

In [55]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 37

# connnexion à la base de données
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
# rc


In [56]:
print(rc[2] +  "\n-----\n" + rc[4] +  "\n-----\n" + rc[5])

Chercher les personnes ayant été influencées par les économistes
-----
https://dbpedia.org/sparql
-----
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  yago: <http://dbpedia.org/class/yago/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>

SELECT DISTINCT  ?person  ?influenced
WHERE
  {
    
       {?influenced  dbo:influencedBy  ?person.}
       


    {   { ?person  a                     dbo:Economist }
      UNION
        { ?person  dbp:profession  dbr:Economist }
      UNION
        { ?person  a                     yago:Economist110043643 }
      UNION
        { ?person  dbp:occupation  dbr:Economist }
      UNION
        { ?person  dbp:field  dbr:Economist }
      UNION
        { ?person  dbp:fields  dbr:Economist }
      UNION
        { ?person  dbp:discipline  dbr:Economist }
      UNION
        { ?person  dbo:profession  dbr:Economist }
      UNION
        { ?person  dbo:occupation  dbr:Economist }
      UNION
        { ?person  db

### Exécuter les requêtes SPARQL en utilisant les fonctions-utilisateur

In [57]:
### Exécuter la requête SPARQL enveloppée par cette fonction, 
# elle se trouve dans le biliothèque-utilisateur _sparql_functions.py_
# le premier paramètre correspond au point d'accès SPARQL, le deuxième à la requête
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [58]:
# Nombre de lignes de la réponse
len(qr['results']['bindings'])

1736

In [59]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr) ]
# r

In [60]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:50]

1736


[['http://dbpedia.org/resource/Aaron_Director',
  'http://dbpedia.org/resource/Steven_N._S._Cheung'],
 ['http://dbpedia.org/resource/Abhijit_Banerjee',
  'http://dbpedia.org/resource/Benjamin_Moll'],
 ['http://dbpedia.org/resource/Abraham_Wald',
  'http://dbpedia.org/resource/Erich_Leo_Lehmann'],
 ['http://dbpedia.org/resource/Abram_Bergson',
  'http://dbpedia.org/resource/Johannes_de_Villiers_Graaff'],
 ['http://dbpedia.org/resource/Adam_Ferguson',
  'http://dbpedia.org/resource/Friedrich_Hayek'],
 ['http://dbpedia.org/resource/Adam_Ferguson',
  'http://dbpedia.org/resource/Georg_Wilhelm_Friedrich_Hegel'],
 ['http://dbpedia.org/resource/Adam_Ferguson',
  'http://dbpedia.org/resource/Jean_Charles_Léonard_de_Sismondi'],
 ['http://dbpedia.org/resource/Adam_Smith',
  'http://dbpedia.org/resource/Antonio_Rosmini'],
 ['http://dbpedia.org/resource/Adam_Smith',
  'http://dbpedia.org/resource/Armen_Alchian'],
 ['http://dbpedia.org/resource/Adam_Smith',
  'http://dbpedia.org/resource/Carl_Menge

In [226]:
# Inspecter les dix premières lignes

def inspect_10_lines (qr):
    i = 0
    for l in qr['results']['bindings']:
        if i < 10:
            print(l)
            i += 1

# inspect_10_lines (qr)

# Transformer les données récupérées et les insérer dans les tables de la base de données SQLite

## Insérer de nouvelles entités

Dans la table _entity_ on insére les entités, i.e. les instances des classes qu'on étudie (personnes, organisations, lieux, objets, etc.)

In [262]:
r[:3]

[['http://dbpedia.org/resource/University_College,_Oxford', '-1.252'],
 ['http://dbpedia.org/resource/University_of_Calgary', '-114.133'],
 ['http://dbpedia.org/resource/University_of_Pittsburgh', '-79.9533']]

In [252]:
## rl = [[l[0], 'https://dbpedia.org/ontology/Person', 'https://dbpedia.org/sparql', 7 ] for l in r]
## rl = [[l[0], 'https://dbpedia.org/ontology/Place', 'https://dbpedia.org/sparql', 26 ] for l in r]
rl = [[l[0], 'https://dbpedia.org/ontology/EducationalInstitution', 'https://dbpedia.org/sparql', 32 ] for l in r]


In [253]:
rl[:3]

[['http://dbpedia.org/resource/A.B.',
  'https://dbpedia.org/ontology/EducationalInstitution',
  'https://dbpedia.org/sparql',
  32],
 ['http://dbpedia.org/resource/Aarhus_University',
  'https://dbpedia.org/ontology/EducationalInstitution',
  'https://dbpedia.org/sparql',
  32],
 ['http://dbpedia.org/resource/Aberystwyth_University',
  'https://dbpedia.org/ontology/EducationalInstitution',
  'https://dbpedia.org/sparql',
  32]]

In [254]:
### Stocker les nouvelles entités en base de données – ne pas oublier d'active le 'commit' !

cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO entity (uri_entity, entity_class, source, fk_query_as_source) VALUES (?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! ##  cn.commit()

cn.close()

### Inspecter le résultat d'une requête après insertion, une fois l'insert ci-dessus effectué

In [180]:
db = 'data/sparql_queries.db'

In [255]:
# connnexion à la base de données
cn = sql.connect(db)
c = cn.cursor()

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c.execute('SELECT count(*) AS eff, entity_class FROM entity GROUP BY entity_class')
result_q = c.fetchall()

# fermer la connexion
cn.close()
result_q[:3]

[(1442, 'https://dbpedia.org/ontology/EducationalInstitution'),
 (6868, 'https://dbpedia.org/ontology/Person'),
 (2773, 'https://dbpedia.org/ontology/Place')]

## Insérer de nouvelles propriétés

Dans la table _property_ on stocke les propriété des entités sous la forme clé-valeur. 


In [61]:
r[:3]

[['http://dbpedia.org/resource/Aaron_Director',
  'http://dbpedia.org/resource/Steven_N._S._Cheung'],
 ['http://dbpedia.org/resource/Abhijit_Banerjee',
  'http://dbpedia.org/resource/Benjamin_Moll'],
 ['http://dbpedia.org/resource/Abraham_Wald',
  'http://dbpedia.org/resource/Erich_Leo_Lehmann']]

In [62]:
# -> (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)
## rl = [[l[0], 'http://dbpedia.org/property/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 10 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/ontology/birthDate', l[1], 'https://dbpedia.org/sparql', 1, 11 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/ontology/birthYear', l[1], 'https://dbpedia.org/sparql', 1, 12 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/ontology/occupation', l[1], 'https://dbpedia.org/sparql', 1, 15 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/ontology/profession', l[1], 'https://dbpedia.org/sparql', 1, 16 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/property/occupation', l[1], 'https://dbpedia.org/sparql', 1, 20 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/property/field', l[1], 'https://dbpedia.org/sparql', 1, 21 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/ontology/field', l[1], 'https://dbpedia.org/sparql', 1, 22 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/ontology/birthPlace', l[1], 'https://dbpedia.org/sparql', 1, 23 ] for l in r]
## rl = [[l[0], 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', l[1], 'https://dbpedia.org/sparql', 1, 27 ] for l in r]
## rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#long', l[1], 'https://dbpedia.org/sparql', 1, 28 ] for l in r]
## rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#lat', l[1], 'https://dbpedia.org/sparql', 1, 29 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/ontology/almaMater', l[1], 'https://dbpedia.org/sparql', 1, 31 ] for l in r]
## rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#long', l[1], 'https://dbpedia.org/sparql', 1, 28 ] for l in r]
## rl = [[l[0], 'http://www.w3.org/2003/01/geo/wgs84_pos#lat', l[1], 'https://dbpedia.org/sparql', 1, 34 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/ontology/influenced', l[1], 'https://dbpedia.org/sparql', 1, 36 ] for l in r]
rl = [[l[0], 'http://dbpedia.org/ontology/influenced', l[1], 'https://dbpedia.org/sparql', 1, 37 ] for l in r]


# -> (uri_entity, property, value, additional_property_1, value_1, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?,?,?)
## rl = [[l[0], 'http://dbpedia.org/property/nationality', l[2], 'lang', l[1], 'https://dbpedia.org/sparql', 1, 13 ] for l in r]
## rl = [[l[0], 'http://dbpedia.org/ontology/nationality', l[2], 'lang', l[1], 'https://dbpedia.org/sparql', 1, 14 ] for l in r]




print(len(rl))
rl[4:8]

1736


[['http://dbpedia.org/resource/Adam_Ferguson',
  'http://dbpedia.org/ontology/influenced',
  'http://dbpedia.org/resource/Friedrich_Hayek',
  'https://dbpedia.org/sparql',
  1,
  37],
 ['http://dbpedia.org/resource/Adam_Ferguson',
  'http://dbpedia.org/ontology/influenced',
  'http://dbpedia.org/resource/Georg_Wilhelm_Friedrich_Hegel',
  'https://dbpedia.org/sparql',
  1,
  37],
 ['http://dbpedia.org/resource/Adam_Ferguson',
  'http://dbpedia.org/ontology/influenced',
  'http://dbpedia.org/resource/Jean_Charles_Léonard_de_Sismondi',
  'https://dbpedia.org/sparql',
  1,
  37],
 ['http://dbpedia.org/resource/Adam_Smith',
  'http://dbpedia.org/ontology/influenced',
  'http://dbpedia.org/resource/Antonio_Rosmini',
  'https://dbpedia.org/sparql',
  1,
  37]]

In [63]:
### OPTIONNEL : si souhaité, stocker la réponse du point d'accès SPARQL dans la table 'result' de la base de données 

cn = sql.connect(db)
c = cn.cursor()

# Insérer les données
c.executemany("INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)", rl)

# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! cn.commit()

cn.close()

### Inspecter le résultat d'une requête après insertion, une fois l'insert ci-dessus effectué

In [54]:
# connnexion à la base de données
cn = sql.connect(db)
c = cn.cursor()

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c.execute('SELECT count(*) AS eff, property FROM property GROUP BY property ORDER BY eff DESC')
result_q = c.fetchall()

# fermer la connexion
cn.close()
result_q[:30]

[(6111, 'http://dbpedia.org/ontology/birthPlace'),
 (5246, 'http://dbpedia.org/ontology/almaMater'),
 (4562, 'http://dbpedia.org/ontology/birthYear'),
 (3837, 'http://dbpedia.org/ontology/birthDate'),
 (3093, 'http://www.w3.org/2003/01/geo/wgs84_pos#long'),
 (3093, 'http://www.w3.org/2003/01/geo/wgs84_pos#lat'),
 (2655, 'http://dbpedia.org/property/nationality'),
 (2301, 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 (2280, 'http://dbpedia.org/property/field'),
 (2104, 'http://dbpedia.org/ontology/influenced'),
 (1897, 'http://dbpedia.org/property/birthDate'),
 (1869, 'http://dbpedia.org/ontology/field'),
 (1809, 'http://dbpedia.org/property/occupation'),
 (1773, 'http://dbpedia.org/ontology/nationality'),
 (1736, 'http://dbpedia.org/ontology/occupation'),
 (620, 'http://dbpedia.org/ontology/profession')]

### Quelles propriétés utiliser ?

Comme indiqué dans le carnet _dbpedia_exploration_ la question se pose de l'articulation entre les propriétés issues de la première méthode d'extraction des données de DBPedia (_http://dbpedia.org/property/..._) et celles de la nouvelle méthode basée sur une ontologie décrite explicititement (_http://dbpedia.org/ontology/..._). Voire les explications fort utiles de cette page:

https://wiki.dbpedia.org/services-resources/ontology

Il faut explorer les données pour comprendre l'information qu'elles contiennent et évaluer son utilité pour répondre aux questionnements.

## Ajout des années de naissance retravaillées

* Dans cette partie, une année de naissance est ajoutée à chaque personne en la calculant à partir des informations disponibles.
* L'année de naissance est stockée dans une propriété existante tout en indiquant sa source: 


In [278]:
db = 'data/sparql_queries.db'

In [279]:
### Inspecter le résultat d'une requête après insertion, une fois l'insert ci-dessus effectué


# noter la logique du regroupement : résultat souhaité = une ligne par personne (URI)

query = """
SELECT GROUP_CONCAT(value), uri_entity, count(*) as eff
FROM property p
WHERE p.property LIKE '%birthD%' OR p.property LIKE '%birthY%'
GROUP BY uri_entity
ORDER BY eff DESC ;

"""

# connnexion à la base de données
cn = sql.connect(db)
c = cn.cursor()

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c.execute(query)
result_q = []
result_q = c.fetchall()

# fermer la connexion
cn.close()
print(len(result_q))
result_q[:5]

3870


[(',1941-02-21,,1941-02-21',
  'http://dbpedia.org/resource/Laurel_Lunt_Prussing',
  4),
 ('1952-05-27,Tokyo Japan,1952-05-27,Tokyo Japan',
  'http://dbpedia.org/resource/Makoto_Yano',
  4),
 (',1940-09-03,,1940-09-03',
  'http://dbpedia.org/resource/Robert_J._Gordon',
  4),
 ('1899-01-17,1899,1899-01-17',
  'http://dbpedia.org/resource/Abram_Lincoln_Harris',
  3),
 ('--06-16,1723-06-05,1723', 'http://dbpedia.org/resource/Adam_Smith', 3)]

In [280]:
### inspecter le résultat
ldn = [l[0] for l in result_q]
print(len(ldn))
ldn.sort()
ldn[:50]

3870


[',--05-07,0007',
 ',1940-09-03,,1940-09-03',
 ',1941-02-21,,1941-02-21',
 ',1949-01-10',
 ',1955-12-14',
 ',1960-10-17',
 ',1962-03-20',
 ',1965-06-30',
 '--01-02',
 '--02-07,--02-07',
 '--02-14,1766-02-14,--02-14',
 '--04-21,--04-21',
 '--06-07,1959-06-07,--06-07',
 '--06-10',
 '--06-16,1723-06-05,1723',
 '--07-18,--07-18',
 '--08-02',
 '--09-03,1957-09-03,--09-03',
 '--10-29',
 '--11-05,--11-05',
 '--12-01',
 '1021-12-08',
 '1031',
 '1332-05-27',
 '1447,1447',
 '1473-02-19',
 '15,15',
 '1560,1560',
 '1571-06-17,1571-06-17',
 '1619-08-29',
 '1621-04-17',
 '1623-05-26,1623',
 '1632-08-29,1632',
 '1635-05-06',
 '1640,1640,1640',
 '1646-02-17,1646-02-17',
 '1648-09-14,1648',
 '1652-09-04,1652',
 '1653-07-05',
 '1657-03-24',
 '1670-11-15,1670',
 '1671-04-21,1671-04-21',
 '1675,1675',
 '1680.0,1680',
 '1694-06-04,1694',
 '1700-02-08',
 '1703,1703',
 '1711-11-19',
 '1712-10-21,1712',
 '1714-09-30,1714']

In [291]:
## extraire seulement les années
listeDateNaiss = []
annee = re.compile(r'\d{4}')

for l in result_q:
    listeDateNaiss.append([annee.findall(l[0]), l[1]])
print(len(listeDateNaiss))
listeDateNaiss[:3]

3870


[[['1941', '1941'], 'http://dbpedia.org/resource/Laurel_Lunt_Prussing'],
 [['1952', '1952'], 'http://dbpedia.org/resource/Makoto_Yano'],
 [['1940', '1940'], 'http://dbpedia.org/resource/Robert_J._Gordon']]

In [293]:
print(len(listeDateNaiss))

#choix de codage: si plusieurs années par personne, retenir la date inférieure
anneeNaiss = [[l[1],int(min(l[0]))] for l in listeDateNaiss if l[0]]
print(len(anneeNaiss))

#trier
anneeNaiss.sort()
anneeNaiss[:5]

3870
3849


[['http://dbpedia.org/resource/A._C._Cuza', 1857],
 ['http://dbpedia.org/resource/A._D._Roy', 1920],
 ['http://dbpedia.org/resource/A._K._Shiva_Kumar', 1956],
 ['http://dbpedia.org/resource/Aaron_Director', 1901],
 ['http://dbpedia.org/resource/Aaron_Farrugia', 1980]]

In [302]:
### Produit avec le script d'insertion ci-dessus

rl = [[l[0], 'http://dbpedia.org/ontology/birthYear', l[1], 'Produit avec script dans carnet dbpedia_production, 8 mai 2021', 1, 0 ] for l in anneeNaiss]


# INSERT OR IGNORE INTO property (uri_entity, property, value, source, quality, fk_query_as_source) VALUES (?,?,?,?,?,?)


print(len(rl))
rl[4:6]

3849


[['http://dbpedia.org/resource/Aaron_Farrugia',
  'http://dbpedia.org/ontology/birthYear',
  1980,
  'Produit avec script dans carnet dbpedia_production, 8 mai 2021',
  1,
  0],
 ['http://dbpedia.org/resource/Aat_van_Rhijn',
  'http://dbpedia.org/ontology/birthYear',
  1892,
  'Produit avec script dans carnet dbpedia_production, 8 mai 2021',
  1,
  0]]